### Reviewing Channel Responses

In [1]:
import numpy as np
import matplotlib as mpl
mpl.rcParams['axes.formatter.useoffset'] = False
import matplotlib.pyplot as plt
import scipy

from scipy.signal import sawtooth, square, savgol_filter
import pandas as pd
import glob as gl
import os
import cmath

from scipy.signal import sawtooth, square,find_peaks
from scipy import spatial
# import lambdafit as lf
from scipy.interpolate import CubicSpline,interp1d
import h5py

from tqdm import tqdm as tqdm_terminal
from tqdm.notebook import trange, tqdm_notebook
from scipy.signal.windows import hann

from scipy.fft import fft, ifft, fftfreq
from copy import deepcopy
from scipy.interpolate import CubicSpline, interp1d
from scipy.optimize import curve_fit

# Best Working Function as of 1/29
(To include easier parameter option for just single channel)

In [4]:
def read_hdf5_chunk_channels(filename, chunk='all', chunk_start=None, chunk_stop=None, single_channel=None):
    # read in file
    file = h5py.File(filename, 'r') 
    # pre-setting range for the for-loop iterating to fix the 0-22 rows of resonator buffer 
    buffer_range_fixed = range(22, (file['time_ordered_data']['adc_i'].shape[0])) 
    # iterate depending on chunk argument 
    if chunk == 'all':
        ch = np.array([channel - 22 for channel in buffer_range_fixed])
        t = np.array(file['time_ordered_data']['timestamp'])
        i = np.array([file['time_ordered_data']['adc_i'][channel] for channel in buffer_range_fixed])
        q = np.array([file['time_ordered_data']['adc_q'][channel] for channel in buffer_range_fixed])
        
    elif chunk == 'some': 
        ch = np.array([channel - 22 for channel in buffer_range_fixed[chunk_start:chunk_stop]])
        t = np.array(file['time_ordered_data']['timestamp'])
        i = np.array([file['time_ordered_data']['adc_i'][channel] for channel in buffer_range_fixed[chunk_start:chunk_stop]])
        q = np.array([file['time_ordered_data']['adc_q'][channel] for channel in buffer_range_fixed[chunk_start:chunk_stop]])
        
    elif chunk == 'single':
        ch = np.array(buffer_range_fixed[single_channel] - 22)
        t = np.array(file['time_ordered_data']['timestamp'])
        i = np.array(file['time_ordered_data']['adc_i'][buffer_range_fixed[single_channel]])
        q = np.array(file['time_ordered_data']['adc_q'][buffer_range_fixed[single_channel]])
        
        
    result = {'channel': ch, 't': t, 'i': i, 'q': q}
                      
    return result

In [5]:
# for graphing all channels, spaced 10000 apart for inspection

def graph_all_channels(channels_data): 
    for i in range(len(channels_data['i'])):
        plt.plot(channels_data['t'],np.abs(channels_data['i'][i]+1j*channels_data['q'][i]) 
             - np.average(np.abs(channels_data['i'][i]+1j*channels_data['q'][i])) + i*10000)

## LO: 4250

In [6]:
%matplotlib qt

# loading files

# time streams
testfile_4250_3432 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_4250.0_20240216162215_t_20240216163432.hd5'
testfile_4250_3956 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_4250.0_20240216162215_t_20240216163956.hd5'
testfile_4250_4412 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_4250.0_20240216162215_t_20240216164412.hd5'

# tone initializations
tone_init_freq_list_4250 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/freq_list_lo_sweep_targeted_1_fcenter_4250.0_20240216162437.npy'
tone_init_sweep_targeted = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_targeted_2_fcenter_4250.0_20240216162651.npy'
#testfile = "/Users/leayamashiro/AliCPT/alicpt_data/data_files/ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5"

# read init files, create csv of values 
freq_list_4250 = np.load(tone_init_freq_list_4250).real # all values still had +0j 
freq_list_4250_table = pd.DataFrame(freq_list_4250)
freq_list_4250_table.to_csv('freq_list_4250.xlsx')

# process data 

data_4250_3432 = read_hdf5_chunk_channels(testfile_4250_3432)
data_4250_3956 = read_hdf5_chunk_channels(testfile_4250_3956)
data_4250_4412 = read_hdf5_chunk_channels(testfile_4250_4412)

In [7]:
# LOOKING AT THE LO SWEEP FILES TO STUDY DEMOD

tone_init_sweep_initial = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_initial_fcenter_4250.0_20240216162224.npy'

init_sweep = np.load(tone_init_sweep_initial)

In [11]:
init_sweep[0]

array([[3.99875000e+09+0.j, 3.99875100e+09+0.j, 3.99875200e+09+0.j, ...,
        3.99924700e+09+0.j, 3.99924800e+09+0.j, 3.99924900e+09+0.j],
       [3.99925098e+09+0.j, 3.99925198e+09+0.j, 3.99925298e+09+0.j, ...,
        3.99974798e+09+0.j, 3.99974898e+09+0.j, 3.99974998e+09+0.j],
       [3.99975195e+09+0.j, 3.99975295e+09+0.j, 3.99975395e+09+0.j, ...,
        4.00024895e+09+0.j, 4.00024995e+09+0.j, 4.00025095e+09+0.j],
       ...,
       [4.50099805e+09+0.j, 4.50099905e+09+0.j, 4.50100005e+09+0.j, ...,
        4.50149505e+09+0.j, 4.50149605e+09+0.j, 4.50149705e+09+0.j],
       [4.50149902e+09+0.j, 4.50150002e+09+0.j, 4.50150102e+09+0.j, ...,
        4.50199602e+09+0.j, 4.50199702e+09+0.j, 4.50199802e+09+0.j],
       [4.50200000e+09+0.j, 4.50200100e+09+0.j, 4.50200200e+09+0.j, ...,
        4.50249700e+09+0.j, 4.50249800e+09+0.j, 4.50249900e+09+0.j]])

In [14]:
# graph data 

#graph_all_channels(data_4250_3432)
#graph_all_channels(data_4250_3956)
graph_all_channels(data_4250_4412)

## LO: 4750

In [6]:
# loading files

# time streams
testfile_4750_5446 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_4750.0_20240216164622_t_20240216165446.hd5'
testfile_4750_5853 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_4750.0_20240216164622_t_20240216165853.hd5'
testfile_4750_0236 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_4750.0_20240216164622_t_20240216170236.hd5'

# tone initializations
tone_init_freq_list_4750 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4750.0_20240216164622/freq_list_lo_sweep_targeted_1_fcenter_4750.0_20240216164814.npy'
#tone_init_sweep_targeted = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_targeted_2_fcenter_4250.0_20240216162651.npy'
#testfile = "/Users/leayamashiro/AliCPT/alicpt_data/data_files/ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5"

# read init files, create csv of values 
freq_list_4750 = np.load(tone_init_freq_list_4750).real # all values still had +0j 
freq_list_4750_table = pd.DataFrame(freq_list_4750)
freq_list_4750_table.to_csv('freq_list_4750.xlsx')

# process data 

data_4750_5446 = read_hdf5_chunk_channels(testfile_4750_5446)
data_4750_5853 = read_hdf5_chunk_channels(testfile_4750_5853)
data_4750_0236 = read_hdf5_chunk_channels(testfile_4750_0236)


In [8]:
# graph data 
%matplotlib qt

graph_all_channels(data_4750_5446)
#graph_all_channels(data_4750_5853)
#graph_all_channels(data_4750_0236)

## LO: 5250

In [8]:
%matplotlib qt

# loading files

# time streams
testfile_5250_1735 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_5250.0_20240216170412_t_20240216171735.hd5'
testfile_5250_2054 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_5250.0_20240216170412_t_20240216172054.hd5'
testfile_5250_3211= '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_5250.0_20240216170412_t_20240216173211.hd5'

# tone initializations
tone_init_freq_list_5250 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_5250.0_20240216170412/freq_list_lo_sweep_targeted_1_fcenter_5250.0_20240216170918.npy'
#tone_init_sweep_targeted = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_targeted_2_fcenter_4250.0_20240216162651.npy'
#testfile = "/Users/leayamashiro/AliCPT/alicpt_data/data_files/ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5"

# read init files, create csv of values 
freq_list_5250 = np.load(tone_init_freq_list_5250).real # all values still had +0j 
freq_list_5250_table = pd.DataFrame(freq_list_5250)
freq_list_5250_table.to_csv('freq_list_5250.xlsx')

# process data 

data_5250_1735 = read_hdf5_chunk_channels(testfile_5250_1735)
data_5250_2054 = read_hdf5_chunk_channels(testfile_5250_2054)
data_5250_3211 = read_hdf5_chunk_channels(testfile_5250_3211)

In [21]:
# graph data 
%matplotlib qt

#graph_all_channels(data_5250_1735)
#graph_all_channels(data_5250_2054)
graph_all_channels(data_5250_3211)

## LO: 5750

In [15]:
%matplotlib qt

# loading files

# time streams
#testfile_5750_3932 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_5750.0_20240216173351_t_20240216173932.hd5'
testfile_5750_4338 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_5750.0_20240216173351_t_20240216174338.hd5'
testfile_5750_4612 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_5750.0_20240216173351_t_20240216174612.hd5'

# tone initializations
tone_init_freq_list_5750 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_5750.0_20240216173351/freq_list_lo_sweep_targeted_1_fcenter_5750.0_20240216173515.npy'
#tone_init_sweep_targeted = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_targeted_2_fcenter_4250.0_20240216162651.npy'
#testfile = "/Users/leayamashiro/AliCPT/alicpt_data/data_files/ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5"

# read init files, create csv of values 
freq_list_5750 = np.load(tone_init_freq_list_5750).real # all values still had +0j 
freq_list_5750_table = pd.DataFrame(freq_list_5750)
freq_list_5750_table.to_csv('freq_list_5750.xlsx')

# # process data 

# data_5750_3932 = read_hdf5_chunk_channels(testfile_5750_3932)
data_5750_4338= read_hdf5_chunk_channels(testfile_5750_4338)
data_5750_4612 = read_hdf5_chunk_channels(testfile_5750_4612)

In [16]:
# graph all channels 

#graph_all_channels(data_5750_4338)
graph_all_channels(data_5750_4612)


## LO: 6250

In [33]:
%matplotlib qt

# loading files

# time streams
testfile_6250_5814 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_6250.0_20240216174729_t_20240216175814.hd5'

# tone initializations
tone_init_freq_list_6250 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_6250.0_20240216174729/freq_list_lo_sweep_targeted_1_fcenter_6250.0_20240216174925.npy'
#tone_init_sweep_targeted = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_targeted_2_fcenter_4250.0_20240216162651.npy'
#testfile = "/Users/leayamashiro/AliCPT/alicpt_data/data_files/ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5"

# read init files, create csv of values 
freq_list_6250 = np.load(tone_init_freq_list_6250).real # all values still had +0j 
freq_list_6250_table = pd.DataFrame(freq_list_6250)
freq_list_6250_table.to_csv('freq_list_6250.xlsx')

# # process data 

data_6250 = read_hdf5_chunk_channels(testfile_6250_5814)

In [32]:
graph_all_channels(data_6250_5814)

## LO: 6750

In [5]:
# time streams
testfile_6750_0505 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/time_streams/ts_toneinit_fcenter_6750.0_20240216175927_t_20240216180505.hd5'

# tone initializations
tone_init_freq_list_6750 = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_6750.0_20240216175927/freq_list_lo_sweep_targeted_1_fcenter_6750.0_20240216180053.npy'
#tone_init_sweep_targeted = '/Users/leayamashiro/AliCPT/alicpt_data/chopped_data/tone_inits/fcenter_4250.0_20240216162215/lo_sweep_targeted_2_fcenter_4250.0_20240216162651.npy'
#testfile = "/Users/leayamashiro/AliCPT/alicpt_data/data_files/ts_toneinit_fcenter_4250.0_20240506174818_t_20240506191017.hd5"

# read init files, create csv of values 
freq_list_6750 = np.load(tone_init_freq_list_6750).real # all values still had +0j 

# read init files, create csv of values 
freq_list_6750 = np.load(tone_init_freq_list_6750).real # all values still had +0j 
freq_list_6750_table = pd.DataFrame(freq_list_6750)
freq_list_6750_table.to_csv('freq_list_6750.xlsx')

data_6750_0505 = read_hdf5_chunk_channels(testfile_6750_0505)

In [6]:
# graph channels

graph_all_channels(data_6750_0505)